<a href="https://colab.research.google.com/github/lakshit2808/Kaggle-Competitions/blob/master/olympics2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Olympics 2021 Prediction

## Importing Libraries

In [347]:
import pandas as pd
import numpy as np

## Importing Dataset

In [348]:
df = pd.read_csv('athletes.csv')


## Data Preprocessing

In [349]:
# Deleting unnecessary columns
del df['dob'], df['id'], df['name']
df['nationality'] = df['nationality'].astype(str)
df.head()

,nationality,sex,height,weight,sport,gold,silver,bronze
0,ESP,male,1.72,64.0,athletics,0,0,0
1,KOR,female,1.68,56.0,fencing,0,0,0
2,CAN,male,1.98,79.0,athletics,0,0,1
3,MDA,male,1.83,80.0,taekwondo,0,0,0
4,NZL,male,1.81,71.0,cycling,0,0,0


### Handling Missing Values

In [350]:
df.isnull().sum()

nationality      0
sex              0
height         330
weight         659
sport            0
gold             0
silver           0
bronze           0
dtype: int64

In [351]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df.iloc[:,2:4])
df.iloc[:,2:4] = imputer.transform(df.iloc[:,2:4])

In [352]:
df.isnull().sum()

nationality    0
sex            0
height         0
weight         0
sport          0
gold           0
silver         0
bronze         0
dtype: int64

In [353]:
X = df.iloc[:, 0:5].values
Y = df.iloc[:,[5,6,7]].values

### Handling Categorical Data

#### Nationality

In [354]:
print(len(df.nationality.unique()))
len(df.sport.unique())

207


28

In [355]:
X = X.transpose()

In [356]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[0]= le.fit_transform(X[0])

In [357]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       ['male', 'female', 'male', ..., 'male', 'male', 'male'],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       ['athletics', 'fencing', 'athletics', ..., 'wrestling',
        'weightlifting', 'athletics']], dtype=object)

#### Sports

In [358]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[4]= le.fit_transform(X[4])

In [359]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       ['male', 'female', 'male', ..., 'male', 'male', 'male'],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       [2, 9, 2, ..., 27, 26, 2]], dtype=object)

#### Gender

In [360]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[1] = le.fit_transform(X[1])

In [361]:
X

array([[59, 102, 33, ..., 38, 202, 158],
       [1, 0, 1, ..., 1, 1, 1],
       [1.72, 1.68, 1.98, ..., 1.85, 1.6, 1.85],
       [64.0, 56.0, 79.0, ..., 100.0, 56.0, 70.0],
       [2, 9, 2, ..., 27, 26, 2]], dtype=object)

### Comb Cate Data

In [362]:
gender = df['sex'].values
nationality = df['nationality'].values
sports = df['sport'].values

In [363]:
cat_gen = dict(zip(gender ,X[1]))
cat_nat = dict(zip(nationality, X[0]))
cat_spt = dict(zip(sports, X[4]))

In [364]:
cat_data = [cat_nat, cat_gen, cat_spt]

## Train Test Split

In [369]:
X = X.transpose()

In [421]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

## Model Trainning / Testing

### Multilinear Regression

In [375]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fmit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [394]:
y_pred = linreg.predict(X_test)

In [411]:
from sklearn.metrics import r2_score
r2_score(Y_test, y_pred)

0.17836820616884982

### Random Forest Regression

In [712]:
from sklearn.ensemble import RandomForestRegressor
ranForReg = RandomForestRegressor(n_estimators = 245, random_state = 40, max_depth = 14,min_samples_split = 10)
ranForReg.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=245, n_jobs=None, oob_score=False,
                      random_state=40, verbose=0, warm_start=False)

In [713]:
y_pred = ranForReg.predict(X_test)

In [720]:
from sklearn.metrics import r2_score
r2 = r2_score(Y_test, y_pred)

In [721]:
r2

0.2612262077228933

### Decision Tree

In [423]:
from sklearn.tree import DecisionTreeRegressor
desTree = DecisionTreeRegressor(random_state=0)
desTree.fit(X_train,Y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [401]:
y_pred = desTree.predict(X_test)

In [402]:
from sklearn.metrics import r2_score
r2_score(Y_test, y_pred)

-0.32892897932801834

### Polynomial Regression

In [420]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train) ## Convert X into polynomial(till degree 2)

# Now Implement Linear Regression to this Polynomial X
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [756]:
y_pred = lin_reg_2.predict(poly_reg.fit_transform(X_test))


6924

In [418]:
from sklearn.metrics import r2_score
r2_score(Y_test, y_pred)

0.027243264081029877

# Program

In [830]:
def Inputs(country, gender, height, weight,sport):
  c = cat_data[0].get(country.upper())
  g = cat_data[1].get(gender.lower())
  s = cat_data[2].get(sport.lower())
  h = height
  w = weight
  list_data = [c,g,h,w,s]
  pred = ranForReg.predict([list_data])
  # Gold
  if pred[0][0] < 0.036:
    gold = 0
  elif pred[0][0] > 0.036 and pred[0][0] < 0.12:
    gold = 1
  elif pred[0][0] > 0.12 and pred[0][0] < 0.9:
    gold = 2
  elif pred[0][0] > 0.9:
    gold = '2+'
  # Silver
  if pred[0][1] < 0.3:
    silver = 0
  elif pred[0][1] > 0.3 and pred[0][1] < 0.5:
    silver = 1
  elif pred[0][1] > 0.5:
    silver = '1+'
  # Bronze
  if pred[0][2] < 0.21:
    bronze = 0
  elif pred[0][2] > 0.21 and pred[0][2] < 4.5:
    bronze = 1 
  elif pred[0][2] > 4.5:
    bronze = '1+'

  prediction = [gold, silver, bronze]
  return prediction



In [831]:
Inputs('BRA', 'female',1.82, 70, 'volleyball')

[1, 0, 0]